# 기본전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("db_aws_rhm_dd.csv", encoding = "utf-8-sig")

In [ ]:
def preprocess(df, dfname = "", to_csv = False):
    if type(df) == str: #df대신 df 경로 및 파일명 입력시
        df = pd.read_csv(df, encoding = "utf-8-sig")
    if df.columns[0] == "Unnamed: 0":
        df = df.loc[:, 1:]
    df.columns = [x.replace("a.", "") for x in df.columns] #colnames
    df["yyyymmdd"] = df[df.columns[0]].apply(lambda x: int(x.split(" ")[0].replace("-", "")))
    df = df.loc[:, df.isnull().mean() != 1]
    if to_csv == True:
        df.to_csv(dfname + ".csv", index = False, encoding = "utf-8-sig")
    return df

def daily(df, back, dfname = "", to_csv = False):
    if type(df) == str: #df대신 df 경로 및 파일명 입력시
        df = pd.read_csv(df, encoding = "utf-8-sig")
    final = pd.DataFrame()
    for s in df["stn_id"].unique():
        print(s, " is processing")
        ss = pd.merge(back, df[df["stn_id"] == s], how = "left", on = "yyyymmdd")
        ss["stn_id"] = s
        final = pd.concat([final, ss], axis = 0)
    if to_csv == True:
        final.to_csv(dfname + ".csv", encoding = "utf-8-sig", index = False)
    return final

def after_daily(df, area, dfname = "", to_csv = False):
    if type(df) == str:
        df = pd.read_csv(df)
    if "isnull" in df.columns:
        df.drop("isnull", axis = 1, inplace = True)
    df.drop(df.columns[["Unnamed" in x for x in df.columns]], axis = 1, inplace = True)
    tm = df.columns[["tm" in x for x in df.columns]][0]
    df[tm] = df["yyyymmdd"].apply(lambda x: str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:])
    idx = df.columns[[x not in ["yyyymmdd", tm, "stn_id"] for x in df.columns]]
    print(idx)
    df = pd.merge(df, area, how = "left", on = "stn_id")
    for i in df[df["name"].isnull()]["stn_id"].unique():
        #step 1 :과반수 이상의 열에서 해당 관측소 결측 50% 이상
        #if (np.round(df.query("stn_id == @i")[idx].isnull().mean(), 1) > 0.5).mean() > 0.5:
        #    df = df.query("stn_id != @i").reset_index(drop = True)
        #step 2 : 필요한 기간의 데이터가 없을 경우(특히 타겟시점!)
        un = df.query("stn_id == @i").dropna(subset = idx)[tm].apply(lambda x: x[:4]).unique()
        if "2011" in un and len(un) < 6:
            df = df.query("stn_id != @i").reset_index(drop = True)
        elif len(un) < 4 or "2016" not in un:
            df = df.query("stn_id != @i").reset_index(drop = True)
    if to_csv == True:
        df.to_csv(dfname + ".csv", encoding = "utf-8-sig", index = False)
    return idx, df

In [ ]:
df = preprocess(df)
print(df.isnull().sum())
df

In [ ]:
back_path = "/Users/seong-yeseul/Downloads/pm10_agg_mac.csv" #경로는 알아서,,
back = pd.DataFrame(pd.read_csv(back_path, encoding = "utf-8-sig")["yyyymmdd"].unique(), columns = ["yyyymmdd"])
df = daily(df, back)
print(df.isnull().sum())
df

In [ ]:
df.to_csv("daily_aws_ta.csv", encoding = "utf-8-sig", index = False)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Mapping

In [44]:
area = pd.read_csv("mapping_0712.csv")
df = pd.read_csv("daily_aws_ta.csv")

In [32]:
def after_daily(df, area, dfname = "", to_csv = False):
    if type(df) == str:
        df = pd.read_csv(df)
    df.drop(df.columns[["Unnamed" in x for x in df.columns]], axis = 1, inplace = True)
    tm = df.columns[["tm" in x for x in df.columns]][0]
    df[tm] = df["yyyymmdd"].apply(lambda x: str(x)[:4] + "-" + str(x)[4:6] + "-" + str(x)[6:])
    idx = df.columns[[x not in ["yyyymmdd", tm, "stn_id"] for x in df.columns]]
    print(idx)
    df = pd.merge(df, area, how = "left", on = "stn_id")
    for i in df[df["name"].isnull()]["stn_id"].unique():
        #step 1 :과반수 이상의 열에서 해당 관측소 결측 50% 이상
        if (np.round(df.query("stn_id == @i")[idx].isnull().mean(), 1) > 0.5).mean() > 0.5:
            df = df.query("stn_id != @i").reset_index(drop = True)
        #step 2 : 필요한 기간의 데이터가 없을 경우(특히 타겟시점!)
        un = df.query("stn_id == @i").dropna(subset = idx)[tm].apply(lambda x: x[:4]).unique()
        if "2011" in un and len(un) < 6:
            df = df.query("stn_id != @i").reset_index(drop = True)
        elif len(un) < 4 or "2016" not in un:
            df = df.query("stn_id != @i").reset_index(drop = True)
    if to_csv == True:
        df.to_csv(dfname + ".csv", encoding = "utf-8-sig", index = False)
    return idx, df

In [45]:
idx, df = after_daily(df, area)

Index(['avg_rhm', 'min_rhm', 'min_rhm_hrmt'], dtype='object')


In [47]:
print(df.isnull().sum())
print(df[df["name"].isnull()].shape[0] / df.shape[0])

yyyymmdd             0
tma                  0
stn_id               0
avg_rhm         751571
min_rhm         728893
min_rhm_hrmt    728885
name             74528
address          74528
lat              74528
long             74528
add              74528
dtype: int64
0.047156726768377254


In [48]:
df = df.dropna(subset = ["name"]).reset_index(drop = True)
df.isnull().sum()

yyyymmdd             0
tma                  0
stn_id               0
avg_rhm         721320
min_rhm         699286
min_rhm_hrmt    699278
name                 0
address              0
lat                  0
long                 0
add                  0
dtype: int64

In [53]:
df.to_csv("null_aws_ta.csv", encoding = "utf-8-sig", index = False)

# 결측치 : 거리 기준 평균

In [58]:
#이게 결측치 채우는 코드임!!!
area3 = df[["stn_id", "name", "lat", "long", "add"]].drop_duplicates().reset_index(drop = True)
for col in idx:
    for s in df[df[col].isnull()]["stn_id"].unique():
        latlon = df.query("stn_id == @s")[["lat", "long"]].drop_duplicates().values[0]
        x = (np.cos(latlon[0]) * 6400 * 2 * np.pi / 360) * np.abs(latlon[1] - area3[["long"]])
        y = 111 * np.abs(latlon[0] - area3["lat"])
        distance = np.sqrt((x**2).squeeze() + y**2).sort_values().head(6)
        dd = pd.concat([area3.iloc[distance.index.to_list(), :], pd.DataFrame(distance, columns = ["dist"])], axis = 1)
        #display(dd)
        tg = df[df["stn_id"] == s]
        ss = dd["stn_id"]
        ii = tg[tg[col].isnull()].index
        ymd = df.loc[ii, "yyyymmdd"]
        #display(df.loc[ii, :])
        df.loc[ii, col] = np.nanmean(df.query("stn_id in @ss and stn_id != @s").pivot(index = "yyyymmdd", columns = "stn_id", values = col).query("yyyymmdd in @ymd"), axis = 1)
        #print(np.nanmean(df.query("stn_id in @ss and stn_id != @s").pivot(index = "yyyymmdd", columns = "stn_id", values = col).query("yyyymmdd in @ymd"), axis = 1))
df.isnull().sum()

yyyymmdd        0
tma             0
stn_id          0
avg_rhm         0
min_rhm         0
min_rhm_hrmt    0
name            0
address         0
lat             0
long            0
add             0
dtype: int64

In [59]:
df.to_csv("final_aws_ta.csv", encoding = "utf-8-sig", index = False)